# Dynamics 365 Business Central Trouble Shooting Guide (TSG) - Performance analysis (partner code issues)

This notebook contains Kusto queries that can help getting to the root cause of a performance issue for an environment. 

The notebook highlights issues for object ids in the ranges [50000..99999] and [1 mio..60 mio] and can be used to find performance problems in code in per-tenant extensions or app source extensions.

Each section in the notebook contains links to the performance tuning guide on docs [aka.ms/bcperformance](aka.ms/bcperformance),  links to the documentation of relevant telemetry in [aka.ms/bctelemetry](aka.ms/bctelemetry), as well as Kusto queries that help dive into a specific area (sessions, web service requests, database calls, reports, and page load times).

NB! Some of the signal used in this notebook is only available in newer versions of Business Central, so check the version of your environment if some sections do not return any data. The signal documentation states in which version a given signal was introduced.

## 1. Connect to Application Insights
First you need to set the notebook Kernel to Python3, load the KQLmagic module (did you install it?) and connect to your Application Insights resource (get appid and appkey from the API access page in the Application Insights portal)

In [16]:
# load the KQLmagic module
%reload_ext Kqlmagic

In [17]:
# Connect to the Application Insights API
%kql appinsights://appid='<add app id from the Application Insights portal>';appkey='<add API key from the Application Insights portal>'

## 2. Define filters
This workbook is designed for troubleshooting a single environment. Please provide values for aadTenantId and environmentName: 

In [18]:
aadTenantId = "<Add AAD tenant id here>"
environmentName = "<add environment name here>"

# date filters for the analysis
# use YYYY-MM-DD format for the dates (ISO 8601)
startDate = "2020-09-20"
endDate = "2020-09-27"

# Analyze performance
Now you can run Kusto queries to look for possible root causes for performance issues.

Either click **Run All** above to run all sections, or scroll down to the type of analysis you want to do and manually run queries

## Web service requests

Performance tuning guide: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/performance/performance-developer#writing-efficient-web-services

Web service telemetry docs: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-webservices-trace

KQL samples: https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/WebServiceCalls.kql

In [19]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0006'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 60000000)
| extend category = tostring( customDimensions.category )
| summarize request_count=count() by category, bin(timestamp, 1d)
| render timechart title= 'Number of web service requests by category endpoints (in partner code)'

In [20]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0008'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 60000000)
| extend category = tostring( customDimensions.category )
       , executionTimeInMS = toreal(totimespan(customDimensions.serverExecutionTime))/10000 //the datatype for executionTime is timespan 
| summarize count=count() by executionTime_ms = bin(executionTimeInMS, 100), category
| order by category, executionTime_ms asc
| render columnchart with  (ycolumns = count, series = category, title= 'Execution time (in ms) of web service requests by category (in partner code)' ) 


In [21]:
%%kql
// 
// top 20 most expensive web service calls (partner code)
// 
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0008'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 60000000)
| extend category = tostring( customDimensions.category )
       , endpoint = tostring( customDimensions.endpoint )
       , executionTimeInMS = toreal(totimespan(customDimensions.serverExecutionTime))/10000 //the datatype for executionTime is timespan 
| summarize count=count(), medianExecutionTimeInMS=percentile(executionTimeInMS, 50) by alObjectId, category, endpoint
| order by medianExecutionTimeInMS desc
| limit 20

## Data related

Performance tuning guide: 
* [Efficient data access](https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/performance/performance-developer#efficient-data-access)
* [Avoid locking](https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/performance/performance-application#avoid-locking)

Database telemetry docs: 
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-long-running-sql-query-trace
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-database-locks-trace

KQL samples:
* https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/Long%20Running%20SQL%20Queries.kql
* https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/LockTimeouts.kql


In [22]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0005'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 60000000)
| summarize count() by bin(timestamp, 1d)
| render timechart title= 'Number of long running SQL queries (in partner code)'

In [23]:
%%kql
// 
// top 20 most expensive long running sql queries (in partner code)
// 
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0005'
| extend alObjectId = toint(customDimensions.alObjectId)
       , alObjectName = toint(customDimensions.alObject)
       , executionTimeInMS = toreal(totimespan(customDimensions.executionTime))/10000 //the datatype for executionTime is timespan 
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 60000000)
| summarize count(), avg(executionTimeInMS) by alObjectId
| limit 20

In [24]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0012'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 60000000)
| summarize request_count=count() by bin(timestamp, 1d)
| render timechart title= 'Number of database lock timeouts (in partner code)'

In [25]:
%%kql
// 
// partner code units where database lock timeouts occur
// 
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0012'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 60000000)
| summarize timeout_count=count() by alObjectId
| order by timeout_count

## Reports

Learn more about how to write performant reports here in the performance tuning guide: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/performance/performance-developer#writing-efficient-reports

Report telemetry docs: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-reports-trace

KQL samples: 
* https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/Reports.kql
* https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/PerformanceTuning/ReportExecution.kql

In [26]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0006'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 60000000)
| extend clientType = tostring( customDimensions.clientType )
       , reportName = tostring( customDimensions.alObjectName )
| where reportName <> ''
| summarize count=count() by clientType, bin(timestamp, 1d)
| render timechart title= 'Number of report execution by client/session type (partner reports)'

In [27]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0006'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 60000000)
| extend reportName = tostring( customDimensions.alObjectName )
       , executionTimeInSec = toreal(totimespan(customDimensions.totalTime))/10000000 //the datatype for executionTime is timespan 
| where reportName <> ''
| summarize avg=avg(executionTimeInSec), median=percentile(executionTimeInSec, 50), percentile95=percentile(executionTimeInSec, 95), max=max(executionTimeInSec)   
  by reportName
| order by percentile95
| limit 10
| render columnchart with  (title= 'Execution time stats of (partner) reports by report name (top 10 by 95% percentile)', ytitle='Time (in seconds)' ) 

In [28]:
%%kql
// 
// top 20 most expensive reports (in partner code)
// 
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0006'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 60000000)
| extend reportName = tostring( customDimensions.alObjectName )
       , executionTimeInSec = toreal(totimespan(customDimensions.totalTime))/10000000 //the datatype for executionTime is timespan 
| where reportName <> ''
| summarize avg=avg(executionTimeInSec), median=percentile(executionTimeInSec, 50), percentile95=percentile(executionTimeInSec, 95), max=max(executionTimeInSec)   
  by reportName
| order by percentile95
| limit 20

In [29]:
%%kql
// 
// top 10 reports that have a low sqlratio
// 
// The higher the number called sqlratio is, the more set-based the report is (it tells you how many sql rows are read per sql statement)
//
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0006'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 60000000)
| project numberOfRows = toint(customDimensions.numberOfRows)
, sqlRowsRead = toint( customDimensions.sqlRowsRead )
, sqlExecutes = toint( customDimensions.sqlExecutes )
, alObjectName = tostring( customDimensions.alObjectName )
, alObjectId = tostring( customDimensions.alObjectId )
, extensionPublisher = tostring(customDimensions.extensionPublisher )
| where isnotnull( sqlExecutes ) and isnotnull( sqlRowsRead )
| summarize numberOfReportExecutions=count(), totalRows=sum(numberOfRows), totalSQLExecutes = sum(sqlExecutes), totalSQLRowsRead = sum(sqlRowsRead) by alObjectId, alObjectName
| extend sqlratio = toreal(totalSQLRowsRead)/totalSQLExecutes
, datasetratio = toreal(totalRows)/totalSQLExecutes
| order by sqlratio asc
| limit 10

## Page views


Page view telemetry docs: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-page-view-trace

KQL samples: https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/PageViews.kql

In [30]:
%%kql
//
// Top 10 longest page times (partner-added pages)
// 
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
pageViews
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 60000000)
| extend objectId = tostring(customDimensions.alObjectId)
| summarize median_load_time_in_MS = percentile(duration,50) by pageName=name, objectId
| order by median_load_time_in_MS desc
| limit 10